#Klasyfikacja gatunków muzycznych - modyfikacja datasetu FMA

Autorzy: Jakub Turliński, Damian Kwiek  
Projekt zaliczeniowy z przedmiotu Głębokie Sieci Neuronowe. Semestr 21Z.

Oryginalny dataset [FMA](https://github.com/mdeff/fma)

Klonowanie repozytorium datasetu

In [ ]:
!git clone https://github.com/mdeff/fma.git
%cd fma
!pwd

Cloning into 'fma'...
remote: Enumerating objects: 823, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 823 (delta 6), reused 12 (delta 4), pack-reused 808
Receiving objects: 100% (823/823), 4.08 MiB | 12.60 MiB/s, done.
Resolving deltas: 100% (528/528), done.
/content/fma
/content/fma


Instalacja pakietów

In [ ]:
!pip install python-dotenv
!pip install -r requirements.txt  # throws "ERROR: an Could not find a version that satisfies the requirement tensorflow-gpu==1.0.1 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.6.0, 2.6.1, 2.6.2, 2.7.0rc0, 2.7.0rc1, 2.7.0)"

!apt-get install p7zip-full  # for unzipping the dataset

     |████████████████████████████████| 4.8 MB 5.3 MB/s 
     |████████████████████████████████| 9.2 MB 37.1 MB/s 
     |████████████████████████████████| 53.2 MB 60 kB/s 
     |████████████████████████████████| 158 kB 57.0 MB/s 
     |████████████████████████████████| 8.9 MB 15.6 MB/s 
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.0.1 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.6.0, 2.6.1, 2.6.2, 2.7.0rc0, 2.7.0rc1, 2.7.0)
ERROR: No matching distribution found for tensorflow-gpu==1.0.1
Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


Pobieranie danych datasetu

In [ ]:
%cd data

# Downloading audiodata takes a lot of time. Using fma_metadata is recommended
# In case of server failure, loading local copy is advised
!curl -O https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!curl -O https://os.unil.cloud.switch.ch/fma/fma_small.zip

!echo "f0df49ffe5f2a6008d7dc83c6915b31835dfe733  fma_metadata.zip" | sha1sum -c -
!echo "ade154f733639d52e35e32f5593efe5be76c6d70  fma_small.zip"    | sha1sum -c -

# '!unzip' command throws "error: not enough memory for bomb detection", '!7za x FileName' command used instead

!7za x fma_metadata.zip
!7za x fma_small.zip

%cd ..

/content/fma/data
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  341M  100  341M    0     0  23.0M      0  0:00:14  0:00:14 --:--:-- 25.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7323M  100 7323M    0     0  25.2M      0  0:04:49  0:04:49 --:--:-- 25.8M
fma_metadata.zip: OK
fma_small.zip: OK

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 358412441 bytes (342 MiB)

Extracting archive: fma_metadata.zip
--
Path = fma_metadata.zip
Type = zip
Physical Size = 358412441

  0%      0% 4 - fma_metadata/raw_albums.csv

#Modyfikacja folderu *fma_small*



In [ ]:
import os

import librosa
import numpy
import skimage.io
import pandas as pd

import utils  # module that manages data from the dataset

AUDIO_DIR = "./data/fma_small/"  # the path to a decompressed fma_*.zip
NEW_DIR = "./FMA_spectrograms/"
CSV_DIR = NEW_DIR + "data_annot.csv"

In [ ]:
# convenience function for scaling data
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

# Extracting data as a spectrogram and save to a PNG file
def spectrogram_image(y, sr, out_dir, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=hop_length*2, hop_length=hop_length)
    mels = numpy.log(mels + 1e-9)  # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(numpy.uint8)
    img = numpy.flip(img, axis=0)  # put low frequencies at the bottom in image
    img = 255-img  # invert image brightness - the darker the area the more energy it has

    # save as PNG
    skimage.io.imsave(out_dir, img)

In [ ]:
# Load metadata, create a directory with custom data
tracks = utils.load('./data/fma_metadata/tracks.csv')

os.mkdir(NEW_DIR)

small = tracks['set', 'subset'] <= 'small'
y_small = tracks.loc[small, ('track', 'genre_top')]

sample_rate = 44100
hop_length = 512 # number of samples per time-step in spectrogram
n_mels = 64  # number of bins in spectrogram. Height of an output PNG image
time_steps = 192  # number of time-steps. Width of an output PNG image
start_sample = 0  # first audio sample in each window of the spectrogram
length_samples = time_steps*hop_length  # total number of audio samples in each window of the spectrogram

iter = 0  # convenience iterator

# Create new, balanced directories
for track_id, genre in y_small.iteritems():
    if not os.path.exists(NEW_DIR + genre):
        os.mkdir(NEW_DIR + genre)
    
    # Save spectrograms as .png files
    mp3_filename = utils.get_audio_path(AUDIO_DIR, track_id)
    out_png_filename = NEW_DIR + genre + '/' + str(track_id) + '.png'

    # TODO create mel spectrograms out of mp3 files, save spectrograms to NEW_DIR
    audio_data, sr = librosa.load(mp3_filename, sr=sample_rate, mono=True)

    window = audio_data[start_sample:start_sample+length_samples]
    spectrogram_image(window, sr=sr, out_dir=out_png_filename, hop_length=hop_length, n_mels=n_mels)

    # TODO save annotation data to a csv file
    df = pd.DataFrame([[out_png_filename, genre]])
    with open(CSV_DIR, 'a+', newline='') as f:
        df.to_csv(CSV_DIR, index=False, mode='a', header=False)
        f.close()

    iter = iter + 1
    print(iter, " spectrograms saved out of ", small.shape[0])


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


1  spectrograms saved out of  106574


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


2  spectrograms saved out of  106574


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


...

4469  spectrograms saved out of  106574


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


4470  spectrograms saved out of  106574


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


NoBackendError: ignored

UWAGA! Dataset FMA zawiera błędy w związku z niepoprawną długością niektórych utworów, przez co możliwe było pobranie 4470 utworów z łącznej liczby 8000.

Zapis pobranych danych jako zip

In [ ]:
# Save as .zip file
!zip -r /content/fma/FMA_spectrograms.zip /content/fma/FMA_spectrograms


  adding: content/fma/FMA_spectrograms/ (stored 0%)
  adding: content/fma/FMA_spectrograms/International/ (stored 0%)
  adding: content/fma/FMA_spectrograms/International/18038.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/43030.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/98671.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/44822.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/1082.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/60544.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/42030.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/4073.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/80766.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/52129.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/43518.png (stored 0%)
  adding: content/fma/FMA_spectrograms/International/6387.p